# Minimal model

fork of *Danilo Lessa Bernardineli*'s **The shortest model ever** injected with logic exploring consensus characterized as a simple channel coding problem. Meant as a light intro to cadCAD.

In [1]:
from cadCAD.configuration import Experiment
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
import pandas as pd
import numpy as np

In [2]:
MONTE_CARLO_RUNS = 1000
SIMULATION_TIMESTEPS = 1

n = 7
sys_params = {
    'msg_error_rate': [.1],
    'bad_block_rate': [.1],
    'lie_rate': [.001]
}


genesis_states = {
    'ground_truth': True,
    'consensus': True,
    'was_wrong': False,
    'nodes': [(True, True) for m in range(n)]
}

def get_ground_truth(params, 
                substep, 
                state_history, 
                prev_state, 
                policy_input):
    
    new_value = np.random.rand() > params['bad_block_rate']
    return ('ground_truth', new_value)

def node_action(params, 
                substep, 
                state_history, 
                prev_state, 
                policy_input):
    
    n = len(prev_state['nodes'])
    ground_truth = prev_state['ground_truth']
    msg_error_rate = params['msg_error_rate']
    lie_rate = params['lie_rate']
    new_value = prev_state['nodes'].copy()
    
    for i in range(n):
        if np.random.rand() > msg_error_rate:
            msg_heard = ground_truth
        else:
            msg_heard = not(ground_truth)
            
        if np.random.rand() > lie_rate:
            msg_sent = msg_heard
        else:
            msg_sent = not(msg_heard)
    
        new_value[i] = (msg_heard,msg_sent)
    
    return ('nodes', new_value)

def get_consensus(params, 
                substep, 
                state_history, 
                prev_state, 
                policy_input):
    
    nodes = prev_state['nodes']
    n = len(nodes)
    msg_error_rate = params['msg_error_rate']
    
    msg_errors = [np.random.rand()<msg_error_rate for m in range(n)]
    #print(msg_errors)
    inbound_msgs = [nodes[m][1]^msg_errors[m] for m in range(n)]
    
    new_value = bool(np.mean(inbound_msgs)>.5)
    return ('consensus', new_value)

def check(params, 
                substep, 
                state_history, 
                prev_state, 
                policy_input):
    
    ground_truth = prev_state['ground_truth']   
    consensus = prev_state['consensus']

    new_value= ground_truth^consensus
    
    return ('was_wrong', new_value)


partial_state_update_blocks = [
    {
        'policies': {
            
        },
        'variables': {
            'ground_truth': get_ground_truth
        }
    },
    {
        'policies': {
            
        },
        'variables': {
            'nodes': node_action
        }
    },
    {
        'policies': {
            
        },
        'variables': {
            'consensus': get_consensus
        }
    },
    {
        'policies': {
            
        },
        'variables': {
            'was_wrong': check
        }
    }
    
]

sim_config = {
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': sys_params
}

In [3]:
sim_params = config_sim(sim_config)

exp = Experiment()
exp.append_configs(
    sim_configs=sim_params,
    initial_state=genesis_states,
    partial_state_update_blocks=partial_state_update_blocks
)

from cadCAD import configs

exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)

simulation = Executor(exec_context=local_mode_ctx,
                      configs=configs)
raw_system_events, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1, 3, 1000, 4)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Total execution time: 5.76s


In [4]:
df = pd.DataFrame(raw_system_events).query("substep==4")
#df.set_index('timestep')['something'].plot()

In [5]:
df['lied'] = df.nodes.apply(lambda x: [not(v[0]==v[1]) for v in x])
df['lie_count'] = df.lied.apply(sum)

In [6]:
df

,ground_truth,consensus,was_wrong,nodes,simulation,subset,run,substep,timestep,lied,lie_count
4,True,True,False,"[(True, True), (True, True), (True, True), (Tr...",0,0,1,4,1,"[False, False, False, False, False, False, False]",0
9,True,True,False,"[(True, True), (True, True), (True, True), (Fa...",0,0,2,4,1,"[False, False, False, False, False, False, False]",0
14,True,True,False,"[(True, True), (True, True), (True, True), (Tr...",0,0,3,4,1,"[False, False, False, False, False, False, False]",0
19,True,True,False,"[(True, True), (True, True), (True, True), (Tr...",0,0,4,4,1,"[False, False, False, False, False, False, False]",0
24,True,True,False,"[(False, False), (True, True), (True, True), (...",0,0,5,4,1,"[False, False, False, False, False, False, False]",0
...,...,...,...,...,...,...,...,...,...,...,...
4979,True,True,False,"[(True, True), (False, False), (False, False),...",0,0,996,4,1,"[False, False, False, False, False, False, False]",0
4984,True,True,False,"[(True, True), (True, True), (True, True), (Tr...",0,0,997,4,1,"[False, False, False, False, False, False, False]",0
4989,False,False,False,"[(False, False), (False, False), (False, False...",0,0,998,4,1,"[False, False, False, False, False, False, False]",0
4994,True,True,False,"[(True, True), (True, True), (True, True), (Tr...",0,0,999,4,1,"[False, False, False, False, False, False, False]",0


In [7]:
df.was_wrong.mean()

0.031

easy Todo
* factor random variables into policy layers
* make the number of nodes a parameter

medium todo
* add economic rewards and penalties
* accumulate rewards over multiple rounds
* compute time discount cashflow model to determine utility for good actors (even after accidental mistakes)

hard todo
* introduce events which create payoffs for lying (inject bad block and try to get it past)
* compute expected utilities of being honest and for colluding to push through the bad block
* introduce strategic cheating if the payoffs for cheating exceed payoff for being honest
* run a range of paratemer sweeps to see the conditions underwhich the cheating appears (and is successful)